In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .appName('init_env') \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
# Criar Schemas
schema_list = ['bronze','silver','gold']

for schema in schema_list:
    spark.sql(f"""
        CREATE DATABASE IF NOT EXISTS iceberg.{schema} COMMENT '' LOCATION 's3a://datalake/iceberg/{schema}/'
    """)

In [ ]:
### Listar catalogs
spark.sql("SHOW databases").show()

In [ ]:
### Criar tabela teste
query = f"""CREATE TABLE IF NOT EXISTS iceberg.bronze.nyc_taxis
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);"""
spark.sql(query)

In [ ]:
### Listar tablels em um schema
spark.sql("SHOW TABLES IN iceberg.bronze").show()


In [ ]:
## Inserir valores
query = f"""INSERT INTO iceberg.bronze.nyc_taxis
VALUES (2, 1000371, 1.8, 15.32, 'Y'), (3, 1000372, 2.5, 22.15, 'N'), (5, 1000373, 0.9, 9.01, 'N'), (6, 1000374, 8.4, 42.13, 'Y');"""

display(spark.sql(query))

In [ ]:
## Consultar dado
spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis where store_and_fwd_flag ='Y' ").show()

In [ ]:
partitions = spark.sql("SELECT * FROM iceberg.bronze.nyc_taxis.history").show()


In [ ]:
spark.stop()